# Exp with finite-size

In [28]:
from isingchat import ising
import numpy as np
import scipy

In [29]:
temp = 1
mag_field = 0
interactions=1/np.array([1,2])**2
num_neighbors = 2
num_tm_eigvals = None

In [9]:
interactions

array([1.  , 0.25])

In [30]:
"""Calculate the Helmholtz free energy for a finite chain."""
nnz_elems, nnz_rows, nnz_cols = ising._csr_finite_log_transfer_matrix_parts_fast(
    temp, mag_field, interactions, num_neighbors
)

# Normalize nonzero matrix elements.
max_w_log_elem = np.max(nnz_elems)
nnz_elems -= max_w_log_elem
norm_nnz_elems = np.exp(nnz_elems)
# Construct the sparse matrix.
num_rows = 2 ** num_neighbors
w_shape = (num_rows, num_rows)
w_matrix = ising.csr_matrix(
    (norm_nnz_elems, (nnz_rows, nnz_cols)), shape=w_shape
)
# Strictly, we should calculate all the eigenvalues and calculate the
# Free energy according to F. A, Kassan-ogly (2001),
#   https://www.tandfonline.com/doi/abs/10.1080/0141159010822758.
# However, in practice, the contribution of the second largest and
# subsequent eigenvalues to the partition function decreases fast, so it
# is sufficient to calculate only a few of the largest eigenvalues.
if num_tm_eigvals is None:
    num_eigvals = min(num_neighbors ** 2, num_rows - 2)
else:
    num_eigvals = min(num_tm_eigvals, num_rows - 2)
# For three or two interactions we take all eigenvalues
if len(interactions) <= 3:
    w_matrix_dense = w_matrix.todense()
    w_all_norm_eigvals: np.ndarray = scipy.linalg.eig(w_matrix_dense)
    w_norm_eigvals = w_all_norm_eigvals[0]
else:
    w_norm_eigvals: np.ndarray = ising.sparse_eigs(
        w_matrix, k=num_eigvals, which="LM", return_eigenvectors=False
    )

In [24]:
eigenvals= ising.sparse_eigs(
        w_matrix, k=num_eigvals, which="LM", return_eigenvectors=False
    )

In [25]:
eigenvals

array([1.+0.j, 1.+0.j])

In [16]:
print(w_matrix)
print("\n")
print(w_norm_eigvals)

  (0, 0)	1.0
  (1, 2)	0.1353352832366127
  (2, 1)	0.1353352832366127
  (3, 3)	1.0


[ 0.13533528+0.j -0.13533528+0.j  1.        +0.j  1.        +0.j]


In [31]:
w_norm_eigvals.real

array([ 0.13533528, -0.13533528,  1.        ,  1.        ])

In [26]:
eigvals_norms: np.ndarray = np.abs(w_norm_eigvals)
max_eigval_norm_idx = eigvals_norms.argmax()
max_eigval_norm = eigvals_norms[max_eigval_norm_idx]
reduced_eigvals = w_norm_eigvals.real / max_eigval_norm
reduced_eigvals_contrib = np.sum(reduced_eigvals ** (num_neighbors))

helm_free_erg = -temp * (
    max_w_log_elem
    + np.log(max_eigval_norm)
    + np.log(reduced_eigvals_contrib.real) / num_neighbors
)

In [27]:
helm_free_erg

-1.6056485542388774